# BiOrthogonal Basis Decomposistion

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tomotok.core.geometry import sparse_line_3d, generate_los, RegularGrid
from tomotok.core.phantoms import gauss_iso
from tomotok.core.inversions import SimpleBob

### Phantom and Grid

In [ ]:
nr, nz = 25, 30
grid = RegularGrid(nr, nz, (.2, .7), (-.3, .3))
phantom = gauss_iso(grid.nr, grid.nz, cen=.3, w=.05) * 2000

In [ ]:
plt.figure()
plt.imshow(phantom, origin='lower', extent=grid.extent)
plt.colorbar()
plt.xlabel('r [m]')
plt.ylabel('Z [m]')
plt.show()

### Artificial Camera Setup

In [ ]:
pinhole = (.8, -.1, .2)
# resolution = (40, 50)
resolution = (40, 40)
# fov = (30, 60)
fov = (40, 40)

npix = resolution[0] * resolution[1]
axis = (-1, -.25, -.2)

In [ ]:
start, end = generate_los(pinhole=pinhole, num=resolution, fov=fov, axis=axis, elong=3)
# nch = start.shape[0]
xch = np.zeros((npix, 2))
ych = np.zeros((npix, 2))
zch = np.zeros((npix, 2))
xch[:, 0] = start[:, 0]
xch[:, 1] = end[:, 0]
ych[:, 0] = start[:, 1]
ych[:, 1] = end[:, 1]
zch[:, 0] = start[:, 2]
zch[:, 1] = end[:, 2]

### Geometry Matrix

In [ ]:
gmat = sparse_line_3d(xch, zch, grid, ych, rmin=.2, step=0.005)

### Synthetic Image

In [ ]:
image = gmat.dot(phantom.reshape(-1, 1))
plt.figure()
plt.axis('off')
plt.imshow(image.reshape(resolution))
plt.colorbar()
plt.show()

## Inversion

In [ ]:
bob = SimpleBob()

In [ ]:
bob.decompose(gmat)

In [ ]:
res1 = bob.dec_mat.T.dot(image)

In [ ]:
plt.imshow(res1.reshape(grid.shape), origin='lower')
plt.colorbar()
# plt.savefig('figure location')
plt.show()

In [ ]:
res2 = bob(image, gmat)

In [ ]:
plt.imshow(res2.reshape(grid.shape), origin='lower')
plt.colorbar()
plt.show()

In [ ]:
rf = gmat.dot(res1)

In [ ]:
plt.imshow((rf-image).reshape(resolution))
plt.colorbar()
plt.show()

## Thresholded

In [ ]:
rest = bob.thresholding(image)

In [ ]:
plt.figure()
plt.imshow(rest.reshape(grid.shape), origin='lower', extent=grid.extent )
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(phantom - rest.reshape(grid.shape), origin='lower', extent=grid.extent)
plt.colorbar()
plt.show()

In [ ]:
rft = gmat.dot(rest)

In [ ]:
plt.imshow((rft-image).reshape(resolution))
plt.colorbar()
plt.show()

### Total Recorded Intensity Check

In [ ]:
print('thresholded {}, full {}, image {} '.format(rft.sum(), rf.sum(), image.sum()))

## Saving decomposed matrices

In [ ]:
# bob.save_decomposition('path/to/decomposed_mat', normalised=False)

In [ ]:
# bob.save_decomposition('path/to/decomposed_normalised_mat', normalised=True)